Syntax natürlicher Sprachen, WS 2023/24

# Klausur

- **Datum: 13.02.2024**
- **Bearbeitungszeitraum: 10:00-11:30**
- **Abgabezeitraum in Moodle: 11:30-11:45**

- Die Klausur umfasst ***20 Einzelaufgaben***, verteilt auf 12 Themengebiete.
- Jede korrekt gelöste Teilaufgabe (`#TO DO`-Codezelle) wird mit 2 Punkten bewertet.

## Übersicht

- [Hinweise zur Bearbeitung](#Hinweise-zur-Bearbeitung)
- [Laden von Paketen](#Laden-von-Paketen)

---
1. [Konstituenten- und Adjunkttests](#1.-Konstituenten--und-Adjunkttests) (4 Punkte)

---
2. [Konstituentengrammatik](#2.-Konstituentengrammatik) (2 Punkte)
3. [X-Bar-Struktur](#3.-X-Bar-Struktur) (4 Punkte)
4. [Parsing-Algorithmen und Rekursionstypen](#4.-Parsing-Algorithmen-und-Rekursionstypen) (2 Punkte)

---
5. [Dependenzstruktur](#5.-Dependenzstruktur) (4 Punkte)
6. [Syntaktische Funktion](#6.-Syntaktische-Funktion) (2 Punkte)
7. [Komplexer Satz](#7.-Komplexer-Satz) (4 Punkte)

---
8. [Unifikation und Subsumption](#8.-Unifikation-und-Subsumption) (4 Punkte)
9. [Feature-based-Grammar](#9.-Feature-based-Grammar) (4 Punkte)

---
10. [Statistisches Parsing](#10.-Statistisches-Parsing) (4 Punkte)
11. [Datengestuetzte Syntaxanalyse](#11.-Datengestuetzte-Syntaxanalyse) (4 Punkte)
12. [Chunk-Analysen](#12.-Chunk-Analysen) (2 Punkte)


---

## Hinweise zur Bearbeitung

### Falls noch nicht geschehen, benennen Sie bitte zunächst die Datei der Klausur-Angabe nach folgendem Schema um: 

 #### `Nachname_Vorname_Matrikelnummer.ipynb`
 

--- 

## Hinweise zum Ausfüllen der Codezellen

### Wenn gegeben, führen Sie zunächst die mit `#RUN` markierten Codezellen zu Beginn einer Aufgabe aus (dies ist für eine erfolgreiche Bearbeitung der Aufgabe notwendig).

### Verändern Sie nur die `#TO DO`-Codezellen (nur gemäß der Angabe in der jeweiligen Aufgabe)!

### Führen Sie `#TO DO`-Codezellen nach Bearbeitung aus, um das Output ihrer Lösung zu generieren (dieses muss als Teil Ihrer Lösung mit abgespeichert werden); bei aufeinander aufbauenden Aufgaben (`a)`, `b)`usw.) ist zudem  notwendig, dass Sie Ihre Lösung aus der vorangehenden Teilaufgabe ausführen, damit diese in der folgenden zur Verfügung steht.

### Angegebene Inhalte (Grammatikregeln usw.) dürfen nicht auskommentiert oder gelöscht werden, außer dies wird explizit anders erwähnt!


### WICHTIG: Setzen Sie den Status des Notebooks ggf. auf `Trusted`, damit alle angegebenen Outputs korrekt angezeigt werden.

---

[zurück zur Übersicht](#Übersicht)


---

## Laden von Paketen

### Führen Sie zu Beginn folgende Codezelle aus.

### Das erfolgreiche Ausführen dieser ist Voraussetzung für die Bearbeitung der folgenden Aufgaben.  

In [1]:
#RUN (Führen Sie diese Code-Zelle aus:)
import nltk
from nltk.tree import Tree
from nltk import FeatStruct
import itertools


from spacy import displacy

def transform_nr_conll(sent_nr):
    sent_list = []
    for line in list(filter(None, sent_nr.split("\n"))):
        line_list = line.split()
        line_list.pop(0)
        line_list.insert(1,"_")
        sent_list.append(" ".join([i for i in line_list[0:]]))

    return "\n".join([i for i in sent_list[0:]])



from nltk import DependencyGraph
from itertools import chain

def _tree_labeled(self, i):
        node = self.get_by_address(i)
        word = node["word"]
        rel = node["rel"]        
        deps = sorted(chain.from_iterable(node["deps"].values()))

        if deps:
            return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])
        else:
            return word+'('+rel+')'
        
def tree_labeled(self):
        node = self.root

        word = node["word"]
        rel = node["rel"]
        deps = sorted(chain.from_iterable(node["deps"].values()))
        return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])

DependencyGraph._tree_labeled = _tree_labeled
DependencyGraph.tree_labeled = tree_labeled



def displacy_dep_input(sent):
    deps = []
    for dep in sent.split('\n'):
        deps.append(dep.split())

    deps = [x for x in deps if x]

    ex = []
    word_list = []
    arc_list = []

    for index, dep in enumerate(deps):
        word_list.append({"text": dep[0], "tag": ""})
        line = index+1
        head = int(dep[2])
        label = dep[3]
        if head>line:
            start = index
            end = head-1
            direction = "left"
        else:
            start = head-1
            end = index  
            direction = "right"
        if(label.lower() != "root"):
            arc_list.append({"start": start, "end": end, "label": label, "dir": direction})

    ex.append({
        "words": word_list,
        "arcs": arc_list
    })    

    return ex



from nltk.featstruct import Feature, UnificationFailure, FeatStructReader
import itertools
from collections import defaultdict


def check_sanity_constraints(th):
    for type1, type2 in itertools.product(th, th):
        if type1 in th[type2] and type2 in th[type1]:
            if type1 != type2:
                raise ValueError(
                    "The type hierarchy is not antisymmetric! " +
                    "{} subsumes {} and vice versa!".format(
                        type1, type2
                    )
                )


def refl_trans_closure(type_hierarchy):
    # make everything a set
    # and compute reflexive closure
    closure = defaultdict(set)
    for t in type_hierarchy:
        closure[t] = set(type_hierarchy[t])
        closure[t].add(t)

    # compute transitive closure
    still_changes = True
    while still_changes:
        still_changes = False
        for x in closure:
            new_for_x = set()
            for y in closure[x]:
                for z in closure[y]:
                    new_for_x.add(z)
            len_before = len(closure[x])
            closure[x].update(new_for_x)
            still_changes |= len(closure[x]) > len_before

    return closure


class HierarchicalFeature(Feature):
    def __init__(self, name, type_hierarchy, **kwargs):
        super(HierarchicalFeature, self).__init__(name, **kwargs)

        self.hierarchy = refl_trans_closure(type_hierarchy)
        check_sanity_constraints(self.hierarchy)

    def unify_base_values(self, fval1, fval2, bindings):
        candidates = self.hierarchy[fval1].intersection(self.hierarchy[fval2])
        score = {t: 0 for t in candidates}
        for type1, type2 in itertools.product(candidates, candidates):
            if type1 in self.hierarchy[type2]:
                score[type1] += 1

        return min(candidates, key=score.__getitem__, default=UnificationFailure)

---

[zurück zur Übersicht](#Übersicht)


--- 

## 1. Konstituenten- und Adjunkttests

## 1.1 Substitutionstest

### Ersetzen Sie unter Erhalt der Wohlgeformtheit die verschiedenen Satzglieder des folgenden Satzes, so dass ein minimaler 2-Wort-Satz entsteht. Sie können alle Formen der Substitution anwenden, also auch Pronominalisierung oder Eliminierung (Ersatz mit leerem String).


In [2]:
#TO_DO:
sentence = ["der kleine Hund", "läuft", "den ganzen Tag", "im Park"]
sentence[0] = "er"
sentence[1] = "läuft"
sentence[2] = ""
sentence[3] = ""
sentence

['er', 'läuft', '', '']

---

[zurück zur Übersicht](#Übersicht)


---

## 1.2 Adjunkt-Test

### Geben Sie über den Listenindex (`TODO`) ein Satzglied des folgenden Satzes an, das vom geschehens-Test als Adjunkt festgestellt wird.

In [3]:
#TO_DO:
sentence = ["der Hund", "hört", "auf der Straße", "nicht", "auf seinen Besitzer"]
satzglied_index = 4
satzglied = sentence.pop(satzglied_index)
print(*sentence, ", und das geschieht ", satzglied)

der Hund hört auf der Straße nicht , und das geschieht  auf seinen Besitzer


---

[zurück zur Übersicht](#Übersicht)


---

## 2. Konstituentengrammatik

### Schreiben Sie zu dem folgenden Beispielsatz für syntaktische Ambiguität ein minimale CFG, die die intendierte Struktur des Beispielsatzes erkennt. Testen Sie anschließend Ihre Grammatik (Ausgabe nur der intendierten Analyse).

- *(time flies)`S` and (mosquitoes like an arrow)`S`*


In [5]:
#TO_DO:
sentence = "time flies and mosquitoes like an arrow"

grammar = nltk.CFG.fromstring("""
    S -> S CConj S
    S -> NP VP
    NP -> N | DET N
    VP -> V | V NP

    N -> "time" | "mosquitoes" | "arrow"
    DET -> "an"
    V -> "flies" | "like"
    CConj -> "and"
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

                         S                            
      ┌──────────┬───────┴────────────┐                
      │          │                    S               
      │          │       ┌────────────┴───┐            
      S          │       │                VP          
 ┌────┴────┐     │       │       ┌────────┴───┐        
 NP        VP    │       NP      │            NP      
 │         │     │       │       │        ┌───┴────┐   
 N         V   CConj     N       V       DET       N  
 │         │     │       │       │        │        │   
time     flies  and  mosquitoes like      an     arrow



---

[zurück zur Übersicht](#Übersicht)


---

## 3. X-Bar-Struktur

### 3a) Gegeben sei folgende koordinierte NP. Schreiben Sie CFG-Regeln, die die hier auftrende Ambiguität im Skopus des Adjektivs über X-Bar-Struktur sichtbar machen.

- *die alten Männer und Frauen*

##### Verwenden Sie NP als Startsymbol in der ersten Zeile der Grammatik.

In [56]:
#TO_DO:
phrase = "die alten Männer und Frauen"

grammar = nltk.CFG.fromstring(
"""
    NP -> DET NOM
    NOM -> NOM CCONJ NOM | ADJ NOM | N

    N -> 'Männer' | 'Frauen'
    ADJ -> 'alten'
    DET -> 'die'
    CCONJ -> 'und'
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(phrase.split()):
    tree.pretty_print(unicodelines=True)    

      NP                         
 ┌────┴─────────┐                 
 │             NOM               
 │         ┌────┴──────┬─────┐    
 │        NOM          │     │   
 │    ┌────┴────┐      │     │    
 │    │        NOM     │    NOM  
 │    │         │      │     │    
DET  ADJ        N    CCONJ   N   
 │    │         │      │     │    
die alten     Männer  und  Frauen

            NP                   
 ┌──────────┴─────┐               
 │               NOM             
 │    ┌───────────┴────┐          
 │    │               NOM        
 │    │     ┌──────────┼─────┐    
 │    │    NOM         │    NOM  
 │    │     │          │     │    
DET  ADJ    N        CCONJ   N   
 │    │     │          │     │    
die alten Männer      und  Frauen



### 3b) Erweitern Sie Ihre Grammatik nun um lexikalische und syntaktische Regeln in X-Bar-Struktur für folgendes Genitiv-NP-Komplement:

- *die alten Männer der Frauen*

In [58]:
#TO_DO:
phrase = "die alten Männer der Frauen"

grammar = nltk.CFG.fromstring(
"""
### GRAMMATIK HERUNTERKOPIEREN!
    NP -> DET NOM
    NOM -> NOM CCONJ NOM | ADJ NOM | N

    N -> 'Männer' | 'Frauen'
    ADJ -> 'alten'
    DET -> 'die'
    CCONJ -> 'und'

######NEUE REGELN:
    NP -> NP NP

    DET -> 'der'
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(phrase.split()):
    tree.pretty_print(unicodelines=True)    

                NP                 
      ┌─────────┴─────────┐         
      NP                  │        
 ┌────┴────┐              │         
 │        NOM             NP       
 │    ┌────┴────┐     ┌───┴────┐    
 │    │        NOM    │       NOM  
 │    │         │     │        │    
DET  ADJ        N    DET       N   
 │    │         │     │        │    
die alten     Männer der     Frauen



---

[zurück zur Übersicht](#Übersicht)


---

## 4. Parsing-Algorithmen und Rekursionstypen


### Geben Sie Regeln an, die ein rekursives center-embedding von Relativsätzen der folgenden Art ermöglichen:

- *My brother opened the window the maid the janitor uncle bill had hired had married had closed.*

##### Sie müssen dabei keine vollständige Grammatik schreiben, sondern nur die Regeln angeben, die die Rekursion ermöglichen. Verwenden Sie dabei nur die Nichtterminale `NP`, `VP`, `RELSATZ`.

In [7]:
#TO_DO:
grammar = nltk.CFG.fromstring("""
    RELSATZ -> NP RELSATZ VP | NP VP
""")

print(grammar)

Grammar with 2 productions (start state = RELSATZ)
    RELSATZ -> NP RELSATZ VP
    RELSATZ -> NP VP


---

[zurück zur Übersicht](#Übersicht)


---

## 5. Dependenzstruktur

## 5.1 Dependenzgrammatik

### Schreiben Sie zu dem folgenden Beispielsatz für syntaktische Ambiguität eine minimale ungelabelte Dependenzgrammatik, die die intendierte Struktur des Beispielsatzes erkennt. Testen Sie anschließend Ihre Grammatik (Ausgabe nur der intendierten Analyse).

- *(time flies and mosquitoes)`NP` (like an arrow)`VP`*

##### Verwenden Sie die UD-Dependenzregeln. Satzzeichen müssen nicht behandelt werden.

In [9]:
#TO_DO:
sentence = "time flies and mosquitoes like an arrow"

grammar = nltk.DependencyGrammar.fromstring("""
    'like' -> 'flies' | 'arrow'
    'arrow' -> 'an'
    'mosquitoes' -> 'and'
    'flies' -> 'time' | 'mosquitoes'
""")

parser = nltk.ProjectiveDependencyParser(grammar)
for tree in parser.parse(sentence.split()):
    print(tree, "\n")
    tree.pretty_print(unicodelines=True)

(like (flies time (mosquitoes and)) (arrow an)) 

              like         
       ┌───────┴────────┐   
     flies              │  
 ┌─────┴───────┐        │   
 │         mosquitoes arrow
 │             │        │   
time          and       an 



---

[zurück zur Übersicht](#Übersicht)


---

## 5.2 Übergangsbasierter Shift-Reduce-Dependency-Parser

#### Gegeben sei folgender Dependenzgraph:

In [10]:
sent_nr = """
1 weil 4 mark 
2 er 4 nsubj
3 ein 4 det
4 Elefant 0 ROOT
5 ist 4 aux
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

### Geben Sie den Typ der REDUCE-Übergänge (`LEFTARC`, `RIGHTARC`) sowie die Reihenfolge deren Durchführung bei Verarbeitung dieser Struktur mit einem Shift-Reduce-Dependency-Parser an.

In [13]:
#TO_DO:
sent_nr = """
1 weil 4 LEFTARC-3 
2 er 4 LEFTARC-2
3 ein 4 LEFTARC-1
4 Elefant 0 ROOT
5 ist 4 RIGHTARC-4
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

---

[zurück zur Übersicht](#Übersicht)


---

## 6. Syntaktische Funktion

### Analysieren Sie die Dependenzbeziehungen des folgenden Satzes im UD-Schema:

- *Des alten Mannes großer Hund lief gestern bei Rot über die Straße.*

#### Verwenden Sie das aus der Vorlesung bekannte Format: 

- pro Zeile: `Position, Wort, Position des Kopfes, Dependenzrelation`
- Wurzelknoten: `Position des Kopfes` = 0, `Dependenzrelation` = ROOT

##### Wie in der Angabe bereits vorgegeben, können Sie auf die Analyse von Satzzeichen verzichten.


In [59]:
#TO_DO:
sent_nr = """
1 Des 3 det
2 alten 3 amod
3 Mannes 5 nmod
4 großer 5 amod
5 Hund 6 nsubj
6 lief 0 ROOT
7 gestern 6 advmod
8 bei 9 case
9 Rot 6 obl
10 über 12 case
11 die 12 det
12 Straße 6 obl
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

tree_labeled = dg.tree_labeled()
tree_labeled.pretty_print(unicodelines=True)  

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

                                                       lief(ROOT)                                          
       ┌──────────────────────────┬────────────────────────┴──────────┬─────────────────────┐               
       │                     Hund(nsubj)                              │                     │              
       │             ┌────────────┴───────────┐                       │                     │               
       │             │                   Mannes(nmod)              Rot(obl)            Straße(obl)         
       │             │            ┌───────────┴────────────┐          │         ┌───────────┴─────────┐     
gestern(advmod) großer(amod)   Des(det)               alten(amod) bei(case) über(case)             die(det)



--- 

[zurück zur Übersicht](#Übersicht)


---

## 7. Komplexer Satz

## 7.1 Analysieren Sie die Dependenzbeziehungen des folgenden Satzes im UD-Schema:

- *Zur selben Zeit wurde Joyce gebissen, weshalb er eine Furcht vor Hunden entwickelte.*

##### Wie in der Angabe bereits vorgegeben, können Sie auf die Analyse von Satzzeichen verzichten.


In [18]:
#TO_DO:
sent_nr = """
1 Zur 3 case
2 selben 3 amod
3 Zeit 6 advmod
4 wurde 6 aux
5 Joyce 6 nsubj
6 gebissen 0 ROOT
7 weshalb 13 mark
8 er 13 nsubj
9 eine 10 det
10 Furcht 13 obj
11 vor 12 case
12 Hunden 10 nmod
13 entwickelte 6 advcl
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

tree_labeled = dg.tree_labeled()
tree_labeled.pretty_print(unicodelines=True)  

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

                                               gebissen(ROOT)                                                              
    ┌───────────┬──────────────────────┬─────────────┴───────────────────────────┐                                          
    │           │                      │                                    entwickelte(                                   
    │           │                      │                                       advcl)                                      
    │           │                      │                            ┌────────────┼──────────────────────┐                   
    │           │                      │                            │            │                 Furcht(obj)             
    │           │                      │                            │            │           ┌──────────┴───────────┐       
    │           │                 Zeit(advmod)                      │            │           │                 Hunden(nmod)
    │

--- 

[zurück zur Übersicht](#Übersicht)


## 7.2 Ergänzen Sie die unten gegebene CFG-Grammatik um syntaktische Regeln für den Beispielsatz:

- *Zur selben Zeit wurde Joyce gebissen, weshalb er eine Furcht vor Hunden entwickelte.*

##### Sie können wieder auf die Analyse von Satzzeichen verzichten.

##### Die Verwendung des X-Bar-Schemas ist nicht notwendig (orientieren Sie sich an den Penn-Treebank-Regeln für komplexe Sätze).  Für den Hauptsatz mit Auxiliarkonstruktion ist bereits eine flache Satzregel vorgegeben, die sie für die Einbettung des Nebensatzes erweitern sollen.

In [22]:
#TO_DO:
sentence = "zur selben Zeit wurde Joyce gebissen weshalb er eine Furcht vor Hunden entwickelte"

grammar = nltk.CFG.fromstring(
    """
    S -> PP AUX NP VP SBAR

###########ERGAENZTE REGELN:
    PP -> P NP
    NP -> DET N | PROPN | PRON | N
    VP -> V
    SBAR -> COMP NP NP PP VP


##Lexikalische Regeln:
    P -> 'zur'
    DET -> 'selben'
    N -> 'Zeit'
    AUX -> 'wurde'
    PROPN -> 'Joyce'
    V -> 'gebissen'
    COMP -> 'weshalb'
    PRON -> 'er'
    DET -> 'eine'
    N -> 'Furcht'
    P -> 'vor'
    N -> 'Hunden'
    V -> 'entwickelte'
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

                                            S                                                   
      ┌───────────────┬─────┬──────┬────────┴────────────────────┐                               
      │               │     │      │                            SBAR                            
      │               │     │      │        ┌─────┬─────────┬────┴─────────┬──────────────┐      
      PP              │     │      │        │     │         │              PP             │     
 ┌────┴─────┐         │     │      │        │     │         │          ┌───┴────┐         │      
 │          NP        │     NP     VP       │     NP        NP         │        NP        VP    
 │    ┌─────┴───┐     │     │      │        │     │    ┌────┴────┐     │        │         │      
 P   DET        N    AUX  PROPN    V       COMP  PRON DET        N     P        N         V     
 │    │         │     │     │      │        │     │    │         │     │        │         │      
zur selben     Zeit wurde

---

[zurück zur Übersicht](#Übersicht)


---

## 8. Unifikation und Subsumption



## 8.1 Unifikation von Merkmalsstrukturen

### Gegeben seien folgende unifizierende Merkmalsstrukturen:

In [23]:
f1 = FeatStruct("[FEAT_a=x,FEAT_b=y]")
f2 = FeatStruct("[FEAT_a=x,FEAT_c=z]")
print(f1.unify(f2))

[ FEAT_a = 'x' ]
[ FEAT_b = 'y' ]
[ FEAT_c = 'z' ]


### Fügen Sie zu der Merkmalsstruktur `f2` ein Feature-Value-Paar hinzu, sodass folgende beide Bedingungen erfüllt sind:

- `f2` unifiziert mit `f1` 
- `f1` subsumiert `f2`.

In [24]:
#TO_DO:
f1 = FeatStruct("[FEAT_a=x,FEAT_b=y]")
f2 = FeatStruct("[FEAT_a=x,FEAT_b=y,FEAT_c=z]")
print(f2.unify(f1))
f1.subsumes(f2)

[ FEAT_a = 'x' ]
[ FEAT_b = 'y' ]
[ FEAT_c = 'z' ]


True

---

[zurück zur Übersicht](#Übersicht)


---

## 8.2 Unifikation mit Typen

### Gegeben sei folgende Agreement-Typhierarchie, die (mit abgekürzten Typnamen) durch das `*AGR*`-Feature implementiert wird:

In [25]:
#RUN:
type_hierarchy = {
    "1": ["1sg", "1du", "1pl"],
    "2": ["2sg", "2pl"],
    "sg": ["1sg", "2sg"],
    "du": ["1du"],
    "pl": ["1pl", "2pl"],
    "1sg": [],
    "1du": [],
    "1pl": [],
    "2sg": [],
    "2pl": []
}
AGR = HierarchicalFeature("AGR", type_hierarchy)
reader = FeatStructReader(features=(AGR,))

### Führen Sie obenstehende Codezelle aus, um die Typhierarchie zu laden.

### Geben Sie eine nicht-leere, von `f1` verschiedene  Merkmalstruktur `f2` an, sodass gilt:

`f1` subsumiert `f2`.

In [27]:
#TO_DO:
f1 = reader.fromstring("[*AGR*='du']")
f2 = reader.fromstring("[*AGR*='1du']")
f1.subsumes(f2)

True

---

[zurück zur Übersicht](#Übersicht)


---

## 9. Feature-based-Grammar


### Gegeben sei folgende CFG-Grammatik:

In [68]:
#RUN (Führen Sie zunächst diese Code-Zelle aus):
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    VP  -> V NP
    VP  -> V
    NP  -> DET N
    NP  -> PRON

    PRON -> 'ich'
    DET -> 'der'
    DET -> 'den'
    N   -> 'Tiger'
    V   -> 'jage'
    V   -> 'jagt'
""")

parser = nltk.ChartParser(grammar,trace=0)

### Die Grammatik erkennt u.a. folgende, grammatisch korrekte Sätze:

In [69]:
sentence = "ich jage den Tiger"
for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

           S               
 ┌─────────┴───┐            
 │             VP          
 │    ┌────────┴───┐        
 NP   │            NP      
 │    │        ┌───┴────┐   
PRON  V       DET       N  
 │    │        │        │   
ich  jage     den     Tiger



In [70]:
sentence = "der Tiger jagt"
for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

          S       
     ┌────┴────┐   
     NP        VP 
 ┌───┴────┐    │   
DET       N    V  
 │        │    │   
der     Tiger jagt



### 9a) Geben Sie einen nach den Regeln der deutschen Grammatik nicht-wohlgeformten Satz an, der fälschlicherweise von der oben angegebenen Grammatik erkannt wird, obwohl er gegen folgendes Constraint verstößt:

- **Subjekt-Verb-Agreement**

In [71]:
#TO_DO:
neg_sentence = "der Tiger jage"
for tree in parser.parse(neg_sentence.split()):
    tree.pretty_print(unicodelines=True)

          S       
     ┌────┴────┐   
     NP        VP 
 ┌───┴────┐    │   
DET       N    V  
 │        │    │   
der     Tiger jage



### 9b) Erweitern Sie die angegebene Grammatik um die notwendigen Merkmale, um die Überproduktion aus 9a) zu verhindern.


In [72]:
#TO_DO:
gramstring = r"""
% start S
    S   -> NP[PER=?x] VP[PER=?x]
    VP[PER=?x]  -> V[PER=?x] NP
    VP[PER=?x]  -> V[PER=?x]
    NP[PER=?x]  -> DET N[PER=?x]
    NP[PER=?x]  -> PRON[PER=?X]

    PRON[PER=1] -> 'ich'
    DET -> 'der'
    DET -> 'den'
    N[PER=3]   -> 'Tiger'
    V[PER=1]   -> 'jage'
    V[PER=3]   -> 'jagt'
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=1)

for tree in parser.parse(neg_sentence.split()):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)

|.der .Tige.jage.|
|[----]    .    .| [0:1] 'der'
|.    [----]    .| [1:2] 'Tiger'
|.    .    [----]| [2:3] 'jage'
|[----]    .    .| [0:1] DET[] -> 'der' *
|[---->    .    .| [0:1] NP[PER=?x] -> DET[] * N[PER=?x] {}
|.    [----]    .| [1:2] N[PER=3] -> 'Tiger' *
|[---------]    .| [0:2] NP[PER=3] -> DET[] N[PER=3] *
|[--------->    .| [0:2] S[] -> NP[PER=?x] * VP[PER=?x] {?x: 3}
|.    .    [----]| [2:3] V[PER=1] -> 'jage' *
|.    .    [---->| [2:3] VP[PER=?x] -> V[PER=?x] * NP[] {?x: 1}
|.    .    [----]| [2:3] VP[PER=1] -> V[PER=1] *


---

[zurück zur Übersicht](#Übersicht)


---
## 10. Statistisches Parsing

## 10.1 PCFG-Gewichte

### Ergänzen Sie in folgender (unvollständigen) PCFG die fehlenden (mit ** markierten) Gewichte:

In [44]:
#TO_DO:
grammar = nltk.PCFG.fromstring("""
    S -> NP VP [0.5]
    S -> VP [0.5]
    NP -> NP SBAR [0.333333]
    NP -> PROPN [0.333333]
    NP -> DET N [0.333333]
    VP -> V [1]
    DET -> 'die' [1]
    N -> 'Katze' [1]
    V -> 'schläft' [1]
""")

sentence = "die Katze schläft"

viterbi_parser = nltk.ViterbiParser(grammar)
for tree in viterbi_parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)
    print(tree)

          S          
     ┌────┴──────┐    
     NP          VP  
 ┌───┴────┐      │    
DET       N      V   
 │        │      │    
die     Katze schläft

(S (NP (DET die) (N Katze)) (VP (V schläft))) (p=0.166666)


---

[zurück zur Übersicht](#Übersicht)


---

## 10.2 Anpassung von PCFG-Gewichten

### In folgender PCFG wird durch teilweise Annotation von Vorfahr-Knoten zwischen NP / DET unter S-Knoten vs. unter VP-Knoten unterschieden.

### Passen Sie die Gewichte im entsprechenden Bereich der PCFG so an, dass die Ableitungswahrscheinlichkeit für die Akkusativ-NP unter S-Knoten minimal wird.

In [73]:
#TO_DO:
grammar = nltk.PCFG.fromstring(
    """
    S   -> NP^S VP [1.0]
    VP  -> V NP^VP [1.0]
    N   -> 'Tiger' [1.0]
    V   -> 'jagt' [1.0]

#AB HIER GEWICHTE ANPASSEN:

    NP^S  -> DET^S N [1.0]
    NP^VP  -> DET^VP N [1.0]

    DET^S -> 'der' [0.99]
    DET^S -> 'den' [0.01]

    DET^VP -> 'den' [0.5]
    DET^VP -> 'der' [0.5]
""")

sents = [
    "der Tiger jagt den Tiger",
    "den Tiger jagt der Tiger"
]

viterbi_parser = nltk.ViterbiParser(grammar)
for sent in sents:
    trees = list(viterbi_parser.parse(sent.split()))
    if trees: [[tree.pretty_print(unicodelines=True), print(tree)] for tree in trees]

                  S                     
       ┌──────────┴─────┐                
       │                VP              
       │          ┌─────┴──────┐         
      NP^S        │          NP^VP      
  ┌────┴─────┐    │     ┌──────┴─────┐   
DET^S        N    V   DET^VP         N  
  │          │    │     │            │   
 der       Tiger jagt  den         Tiger

(S
  (NP^S (DET^S der) (N Tiger))
  (VP (V jagt) (NP^VP (DET^VP den) (N Tiger)))) (p=0.495)
                  S                     
       ┌──────────┴─────┐                
       │                VP              
       │          ┌─────┴──────┐         
      NP^S        │          NP^VP      
  ┌────┴─────┐    │     ┌──────┴─────┐   
DET^S        N    V   DET^VP         N  
  │          │    │     │            │   
 den       Tiger jagt  der         Tiger

(S
  (NP^S (DET^S den) (N Tiger))
  (VP (V jagt) (NP^VP (DET^VP der) (N Tiger)))) (p=0.005)


---

[zurück zur Übersicht](#Übersicht)


---

## 11. Datengestuetzte Syntaxanalyse
   

## 11.1 Lexikalisierte CFG (mit Merkmalen)

### Gegeben sei folgende CFG, die genau eine der möglichen Lesarten dieses ambigen Beispielsatzes erkennt:

- *time flies like an arrow*

In [47]:
sentence = "time flies like an arrow"

grammar = nltk.CFG.fromstring("""
    S -> VP
    NP -> DET N
    NP -> N
    VP -> V NP PP
    PP -> P NP
    
    V -> 'time'
    N -> 'flies'
    P -> 'like'
    DET -> 'an'
    N -> 'arrow'
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)

            S                
            │                 
            VP               
 ┌─────┬────┴────┐            
 │     │         PP          
 │     │    ┌────┴───┐        
 │     NP   │        NP      
 │     │    │    ┌───┴────┐   
 V     N    P   DET       N  
 │     │    │    │        │   
time flies like  an     arrow



### Führen Sie eine entsprechende Kopfannotation über ein `HEAD`-Merkmal durch, die dieser Lesart entspricht.

##### Halten Sie sich dabei an die UD-Dependenzregeln.


In [49]:
#TO_DO:
sentence = "time flies like an arrow"

gramstring = r"""
% start S
    S[HEAD=?v] -> VP[HEAD=?v]
    NP[HEAD=?n] -> DET[]  N[HEAD=?n]
    NP[HEAD=?n] -> N[HEAD=?n]
    VP[HEAD=?v] -> V[HEAD=?v]  NP[]  PP[]
    PP[HEAD=?n] -> P[]  NP[HEAD=?n]
    
    V[HEAD="time"] -> 'time'
    N[HEAD="flies"] -> 'flies'
    P[HEAD="like"] -> 'like'
    DET[HEAD="an"] -> 'an'
    N[HEAD="arrow"] -> 'arrow'
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    print(tree)
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
    #display(tree)

(S[HEAD='time']
  (VP[HEAD='time']
    (V[HEAD='time'] time)
    (NP[HEAD='flies'] (N[HEAD='flies'] flies))
    (PP[HEAD='arrow']
      (P[HEAD='like'] like)
      (NP[HEAD='arrow'] (DET[HEAD='an'] an) (N[HEAD='arrow'] arrow)))))
                                 S[HEAD='time']                                                  
                                       │                                                          
                                VP[HEAD='time']                                                  
      ┌───────────────┬────────────────┴───────────────┐                                          
      │               │                         PP[HEAD='arrow']                                 
      │               │                ┌───────────────┴────────────────┐                         
      │        NP[HEAD='flies']        │                         NP[HEAD='arrow']                
      │               │                │               ┌────────────────┴────────

---

[zurück zur Übersicht](#Übersicht)


---

## 11.2 Parent Annotation (mit Symbolerweiterung)

### Führen Sie in der CFG aus der vorherigen Aufgabe über Symbolerweiterung (mit `^` als Trennerzeichen) eine vollständige *Parent Annotation* durch, wie sie durch die Regelanwendungen im Syntaxbaum der Angabe impliziert ist. Falls notwendig, ergänzen Sie Regeln.

In [52]:
#TO_DO:
sentence = "time flies like an arrow"

grammar = nltk.CFG.fromstring("""
    S -> VP^S
    NP^PP -> DET^NP  N^NP
    NP^VP -> N^NP
    VP^S -> V^VP  NP^VP  PP^VP
    PP^VP -> P^PP  NP^PP
    
    V^VP -> 'time'
    N^NP -> 'flies'
    P^PP -> 'like'
    DET^NP -> 'an'
    N^NP -> 'arrow'

###########ERGAENZTE REGELN:    
  
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

            S                     
            │                      
           VP^S                   
 ┌─────┬────┴─────┐                
 │     │        PP^VP             
 │     │    ┌─────┴──────┐         
 │   NP^VP  │          NP^PP      
 │     │    │     ┌──────┴─────┐   
V^VP  N^NP P^PP DET^NP        N^NP
 │     │    │     │            │   
time flies like   an         arrow



---

[zurück zur Übersicht](#Übersicht)


---

## 12. Chunk-Analysen

### Geben Sie die *IOB-Tag-Sequenz* und *POS-Label* einer NP-PP-VP-Chunk-Analyse für den folgenden Satz an:

- *Ulysses gilt als der bedeutendste Roman des irischen Schriftstellers James Joyce und als richtungsweisend für den modernen Roman.*

In [74]:
#TO_DO:
iob_list = [
("Ulysses", "PROPN", "B-NP"),
("gilt", "V", "B-VP"),
("als", "CCONJ", "O"),
("der", "DET", "B-NP"),
("bedeutendste", "ADJ", "I-NP"),
("Roman", "N", "I-NP"),
("des", "DET", "B-NP"),
("irischen", "ADJ", "I-NP"),
("Schriftstellers", "N", "I-NP"),
("James", "PROPN", "B-NP"),
("Joyce", "PROPN", "I-NP"),
("und", "CCONJ", "O"),
("als", "CCONJ", "O"),
("richtungsweisend", "ADJ", "O"),
("für", "P", "B-PP"),
("den", "DET", "B-NP"),
("modernen", "ADJ", "I-NP"),
("Roman", "N", "I-NP"),
(".", "PUNCT", "O")
]

from nltk import conlltags2tree
tree = conlltags2tree(iob_list)
print(tree)

(S
  (NP Ulysses/PROPN)
  (VP gilt/V)
  als/CCONJ
  (NP der/DET bedeutendste/ADJ Roman/N)
  (NP des/DET irischen/ADJ Schriftstellers/N)
  (NP James/PROPN Joyce/PROPN)
  und/CCONJ
  als/CCONJ
  richtungsweisend/ADJ
  (PP für/P)
  (NP den/DET modernen/ADJ Roman/N)
  ./PUNCT)


---

[zurück zur Übersicht](#Übersicht)
